In [ ]:
# Big Data Machine Learning Classification with Spark

Bu çalışmamızda, Churn verilerine bakarak müsterilerin bizi terk edip etmeyeceğini tahmin eden bir model geliştireceğiz

In [ ]:
#pip install pyspark

In [ ]:
import pandas as pd
import seaborn as sns

In [42]:
df=pd.read_csv('churn.csv')

In [ ]:
df.head()

,Unnamed: 0,Names,Age,Total_Purchase,Account_Manager,Years,Num_Sites,Churn
0,0,Cameron Williams,42.0,11066.80,0,7.22,8.0,1
1,1,Kevin Mueller,41.0,11916.22,0,6.50,11.0,1
2,2,Eric Lozano,38.0,12884.75,0,6.67,12.0,1
3,3,Phillip White,42.0,8010.76,0,6.71,10.0,1
4,4,Cynthia Norton,37.0,9191.58,0,5.56,9.0,1


In [ ]:
df.sample(5)

,Unnamed: 0,Names,Age,Total_Purchase,Account_Manager,Years,Num_Sites,Churn
781,781,Amber Evans,37.0,10348.37,1,7.65,6.0,0
898,898,Justin Leonard,51.0,6517.93,1,5.47,10.0,0
887,887,Kara Anderson,47.0,9182.81,1,6.45,11.0,0
58,58,David Rodriguez,38.0,8642.01,1,4.76,11.0,1
259,259,Paul Wagner,42.0,10427.12,0,6.64,10.0,0


In [ ]:
df.tail()

,Unnamed: 0,Names,Age,Total_Purchase,Account_Manager,Years,Num_Sites,Churn
895,895,Paul Miller,42.0,12800.82,1,3.62,8.0,0
896,896,Natalie Hodges,52.0,9893.92,0,6.91,7.0,0
897,897,Ana Smith,45.0,12056.18,0,5.46,4.0,0
898,898,Justin Leonard,51.0,6517.93,1,5.47,10.0,0
899,899,Joseph Williams,39.0,9315.60,1,5.02,10.0,0


In [ ]:
df.shape

(900, 8)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900 entries, 0 to 899
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       900 non-null    int64  
 1   Names            900 non-null    object 
 2   Age              900 non-null    float64
 3   Total_Purchase   900 non-null    float64
 4   Account_Manager  900 non-null    int64  
 5   Years            900 non-null    float64
 6   Num_Sites        900 non-null    float64
 7   Churn            900 non-null    int64  
dtypes: float64(4), int64(3), object(1)
memory usage: 56.4+ KB


In [ ]:
df.describe()

,Unnamed: 0,Age,Total_Purchase,Account_Manager,Years,Num_Sites,Churn
count,900.000000,900.000000,900.000000,900.000000,900.000000,900.000000,900.000000
mean,449.500000,41.816667,10062.824033,0.481111,5.273156,8.587778,0.166667
std,259.951919,6.127560,2408.644532,0.499921,1.274449,1.764836,0.372885
min,0.000000,22.000000,100.000000,0.000000,1.000000,3.000000,0.000000
25%,224.750000,38.000000,8497.122500,0.000000,4.450000,7.000000,0.000000
50%,449.500000,42.000000,10045.870000,0.000000,5.215000,8.000000,0.000000
75%,674.250000,46.000000,11760.105000,1.000000,6.110000,10.000000,0.000000
max,899.000000,65.000000,18026.010000,1.000000,9.150000,14.000000,1.000000


In [ ]:
df.isnull().sum().sort_values(ascending=True)

Unnamed: 0         0
Names              0
Age                0
Total_Purchase     0
Account_Manager    0
Years              0
Num_Sites          0
Churn              0
dtype: int64

In [ ]:
df.corr(numeric_only=True)

,Unnamed: 0,Age,Total_Purchase,Account_Manager,Years,Num_Sites,Churn
Unnamed: 0,1.000000,-0.037801,-0.018938,-0.067247,-0.143873,-0.378287,-0.645498
Age,-0.037801,1.000000,-0.037208,-0.014749,0.005625,-0.006070,0.085926
Total_Purchase,-0.018938,-0.037208,1.000000,0.015856,-0.005623,-0.003390,0.024031
Account_Manager,-0.067247,-0.014749,0.015856,1.000000,0.022930,0.033401,0.070611
Years,-0.143873,0.005625,-0.005623,0.022930,1.000000,0.051642,0.214329
Num_Sites,-0.378287,-0.006070,-0.003390,0.033401,0.051642,1.000000,0.525398
Churn,-0.645498,0.085926,0.024031,0.070611,0.214329,0.525398,1.000000


In [43]:
df=df.drop('Unnamed: 0',axis=1)
df.head()

,Age,Total_Purchase,Account_Manager,Years,Num_Sites,Churn
0,42.0,11066.80,0,7.22,8.0,1
1,41.0,11916.22,0,6.50,11.0,1
2,38.0,12884.75,0,6.67,12.0,1
3,42.0,8010.76,0,6.71,10.0,1
4,37.0,9191.58,0,5.56,9.0,1


In [50]:
from os import access
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import GBTClassifier

spark = SparkSession.builder.appName('ChurnClassifier').getOrCreate()

data = spark.read.csv('churn.csv', header=True,inferSchema=True)
data = data.drop('Names')#Adlar sütununun veri türü string olduğu için onu attık

feature_columns = data.columns[:-1]
assembler = VectorAssembler(inputCols=feature_columns, outputCol='features')
data = assembler.transform(data).select('features', 'Churn')

train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)

GBT = GBTClassifier(labelCol='Churn', featuresCol='features')
model = GBT.fit(train_data)

predictions = model.transform(test_data)
evaluator = BinaryClassificationEvaluator(labelCol='Churn')
accuracy = evaluator.evaluate(predictions)
print('Accuracy:', accuracy)

Accuracy: 0.9918032786885246
